# Rewrite-Retrieve-Read

**Rewrite-Retrieve-Read** is a method proposed in the paper [Query Rewriting for Retrieval-Augmented Large Language Models](https://arxiv.org/pdf/2305.14283.pdf)

> Because the original query can not be always optimal to retrieve for the LLM, especially in the real world... we first prompt an LLM to rewrite the queries, then conduct retrieval-augmented reading

We show how you can easily do that with LangChain Expression Language

## Baseline

Baseline RAG (**Retrieve-and-read**) can be done like the following:

In [10]:
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.utilities import DuckDuckGoSearchAPIWrapper

In [90]:
template = """Answer the users question based only on the following context:

<context>
{context}
</context>

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(temperature=0)

search = DuckDuckGoSearchAPIWrapper()


def retriever(query):
    return search.run(query)

In [91]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

In [92]:
simple_query = "what is langchain?"

In [93]:
chain.invoke(simple_query)

'LangChain is a powerful framework and versatile Python library that simplifies the development of language-based applications. It offers a suite of features for artificial general intelligence, including document analysis and summarization, as well as the ability to build chatbots that interact with users naturally. It is an open-source library that enables developers and researchers to create, experiment with, and analyze language models and agents. LangChain provides a generic interface to many foundation models, prompt management, and acts as a central interface to other components like prompt templates, other language models, external data, and other tools via agents. Overall, LangChain is designed to help developers build end-to-end applications using language models and offers a range of tools, components, and interfaces to simplify the process.'

While this is fine for well formatted queries, it can break down for more complicated queries

In [94]:
distracted_query = "man that sam bankman fried trial was crazy! what is langchain?"

In [95]:
chain.invoke(distracted_query)

'Based on the given context, there is no information about "langchain."'

This is because the retriever does a bad job with these "distracted" queries

In [96]:
retriever(distracted_query)

'Sam Bankman-Fried, FTX\'s founder, responded with a single word: "Oof.". Less than a year later, Mr. Bankman-Fried, 31, is on trial in federal court in Manhattan, fighting criminal charges ... NEW YORK, Oct 18 (Reuters) - A U.S. judge on Wednesday overruled objections by Sam Bankman-Fried\'s lawyers and allowed jurors in the FTX founder\'s fraud trial to see a profane message he... Business FTX founder Sam Bankman-Fried\'s trial is about to start. Here\'s what you need to know In testimony on Tuesday and Wednesday that got tearful at times, Ellison accused... Sam Bankman-Fried, who was once hailed as a virtuoso in cryptocurrency trading, is on trial over the collapse of FTX, the financial exchange he founded. Bankman-Fried is accused of... Business Oct 2, 2023 11:29 AM The Trial of Sam Bankman-Fried, Explained White-collar defendants use three main defenses: "It wasn\'t me, I didn\'t mean it, and the people that say I did are...'

## Rewrite-Retrieve-Read Implementation

The main part is a rewriter to rewrite the search query

In [107]:
# template = """Provide a better search query for \
# web search engine to answer the given question, end \
# the queries with ’**’. Question: \
# {x} Answer:"""
# prompt = ChatPromptTemplate.from_template(template)

In [109]:
from langchain import hub

prompt = hub.pull("langchain-ai/rewrite")

In [110]:
print(prompt.template)

Provide a better search query for web search engine to answer the given question, end the queries with ’**’.  Question {x} Answer:


In [111]:
# Parser to remove the `**`

def _parse(text):
    return text.strip("**")

In [112]:
rewriter = prompt | ChatOpenAI(temperature=0) | StrOutputParser() | _parse

In [113]:
rewriter.invoke({"x": distracted_query})

'What is the definition and purpose of Langchain?'

In [101]:
rewrite_retrieve_read_chain = (
    {
        "context": {"x": RunnablePassthrough()} | rewriter | retriever,
        "question": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

In [102]:
rewrite_retrieve_read_chain.invoke(distracted_query)

'Based on the given context, LangChain is an open-source framework designed to simplify the creation of applications using large language models (LLMs). It provides a standard interface for chains, integrations with other tools, and end-to-end chains for common applications. LangChain enables LLM models to generate responses based on up-to-date online information and simplifies the organization of large volumes of data for easy access by LLMs. It is an AI framework with unique features that simplify the development of language-based applications.'